In [ ]:
import sys
from astropy.io import fits
import glob
import numpy as np
import time
import resource
import subprocess
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import resource
import gc

In [ ]:
def plotImage(image_data):
    plt.figure()
    plt.imshow(image_data, origin='lower', cmap='magma')
    plt.colorbar(label='Intensidad')
    plt.title('Imagen del Disco Protoplanetario Normalizada')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.savefig('Image-disk.png')
    plt.show()

def plotVelocity(vx, vy, titlex, titley):
    
    # Visualización de las componentes de velocidad
    plt.figure(figsize=(15, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(vx, origin='lower', aspect='auto', cmap='magma')
    plt.colorbar(label='Velocidad')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(titlex)
    plt.axis('equal')
    
    plt.subplot(1, 2, 2)
    plt.imshow(vy, origin='lower', aspect='auto', cmap='magma')
    plt.colorbar(label='Velocidad')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(titley)
    plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig('Velocity-disk.png')
    plt.show()

In [ ]:
def plotVelocityField(X, Y, Vx, Vy, npix):
    speed = np.sqrt(Vx**2 + Vy**2)
    lw = 3.5*speed / speed.max()

    plt.figure(figsize=(6, 5))
    strm = plt.streamplot(X, Y, Vx, Vy, color=speed, linewidth=lw, cmap='magma')
    plt.colorbar(strm.lines, label='Velocidad')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Campo de Velocidad del Disco')
    plt.show()

In [ ]:
def proyectar_velocidades_fargo(file_path_field, out, npix, i_deg, PA_deg):

    # Leer coordenadas y velocidades polares
    r = np.loadtxt(file_path_field + 'domain_y.dat')[3:-4]
    phi = np.loadtxt(file_path_field + 'domain_x.dat')[:-1]
    nr, nphi = len(r), len(phi)

    Vr = np.fromfile(file_path_field + f'gasvy{out:d}.dat', dtype=np.float64).reshape(nr, nphi)
    Vphi = np.fromfile(file_path_field + f'gasvx{out:d}.dat', dtype=np.float64).reshape(nr, nphi)

    # Convertir de coordenadas polares a cartesianas
    Phi, R = np.meshgrid(phi, r)
    X = R * np.cos(Phi)
    Y = R * np.sin(Phi)

    Vx = Vr * np.cos(Phi) - Vphi * np.sin(Phi)
    Vy = Vr * np.sin(Phi) + Vphi * np.cos(Phi)
        
    # Ángulos en radianes
    inc = np.radians(i_deg)  # inclinación
    PAr = np.radians(PA_deg)  # posición del eje mayor (PA)
    
    # Aplicar la inclinación en Y (comprime posiciones y velocidades en ese eje) 
    Y_incl = Y * np.cos(inc)
    Vy_incl = Vy * np.cos(inc)
    
    # Rotar el sistema para alinear el eje mayor con X
    X_rot = X * np.cos(PAr) + Y_incl * np.sin(PAr)
    Y_rot = -X * np.sin(PAr) + Y_incl * np.cos(PAr)
    
    Vx_rot = Vx * np.cos(PAr) + Vy_incl * np.sin(PAr)
    Vy_rot = -Vx * np.sin(PAr) + Vy_incl * np.cos(PAr)
    
    # Interpolación en malla cartesiana uniforme
    new_x = np.linspace(X_rot.min(), X_rot.max(), npix)
    new_y = np.linspace(Y_rot.min(), Y_rot.max(), npix)
    new_X, new_Y = np.meshgrid(new_x, new_y)
    
    Vx_cartesian = griddata((X_rot.ravel(), Y_rot.ravel()), Vx_rot.ravel(), (new_X, new_Y), method='nearest')
    Vy_cartesian = griddata((X_rot.ravel(), Y_rot.ravel()), Vy_rot.ravel(), (new_X, new_Y), method='nearest')
    
    Vx_cartesian = np.flip(np.squeeze(Vx_cartesian), axis=(0,1))
    Vy_cartesian = np.flip(np.squeeze(Vy_cartesian), axis=(0,1))

    if pnt:
        print("Dimensiones Vx, Vy - shape: ", Vx.shape, Vy.shape)
        print("Vx Max: ", Vx.max(), " Vx Min: ", Vx.min())
        print("Vy Max: ", Vy.max(), " Vy Min: ", Vy.min())
        plotVelocity(Vx, Vy, 'Componente de Velocidad X cartesian', \
                     'Componente de Velocidad Y cartesian')

        print("Nuevas dimensiones X, Y - shape: ", new_X.shape, new_Y.shape)
        print("Nuevas dimensiones Vx, Vy - shape: ", Vx_cartesian.shape, Vy_cartesian.shape)
        print("Vx cartesian npix Max: ", Vx_cartesian.max(), " Vx cartesian npix Min: ", Vx_cartesian.min())
        print("Vy cartesian npix Max: ", Vy_cartesian.max(), " Vy cartesian npix Min: ", Vy_cartesian.min())
        print("Number of NaNs in Vy_cartesian:", np.isnan(Vx_cartesian).sum())
        print("Number of NaNs in Vy_cartesian:", np.isnan(Vy_cartesian).sum())

        plotVelocity(Vx_cartesian, Vy_cartesian, 'Componente de Velocidad X cartesian npix', \
                     'Componente de Velocidad Y cartesian npix')

        print("Imprimiendo campo de velocidades")
        plotVelocityField(new_X, new_Y, Vx_cartesian, Vy_cartesian, npix)

    return Vx_cartesian, Vy_cartesian, new_X, new_Y


## Normalización por disco

In [ ]:
def get_numpy_files_norm(simulation, offset, time_step, npix, err_sim, v_time, \
                         base_path_time, file_time, base_path_field,\
                         normtype, incl, rot, origen):

    images_data_norm_tot = []
    vfield_data_norm_x_tot = []
    vfield_data_norm_y_tot = []
    vfield_data_coordinates_tot = []
    tag_data = []

    for sim in range(simulation):
    
        if (sim+offset) in err_sim:
            continue
    
        images_data_norm_by_sim = []
        tag_data.append([f"{file_time}", f"{origen}_sim_{offset + sim}"])
        
        for t in range(1, time_step):
        
            ###TIME####
            # Ruta donde están guardadas las imágenes
            file_path_time = f"{base_path_time}/{file_time}/sim_{sim+offset:01d}/output_{t:01d}/image_out_wl1300.fits"

            hdulist = fits.open(file_path_time, mode='readonly', do_not_scale_image_data=True)
            image_data = hdulist[0].data
            image_data = np.flipud(image_data)
            hdulist.close()
            image_data = np.squeeze(image_data)
            
            if normtype == 'percentil':
                # Normalizar datos utilizando percentiles:
                p2, p98 = np.percentile(image_data, (2, 98))
                image_data_norm = np.clip((image_data - p2) / (p98 - p2), 0, 1)

            elif normtype == 'log':
                # Normalizar datos utilizando transformación logarítmica:
                aux_norm = np.log1p(image_data - image_data.min())
                image_data_norm = np.divide(aux_norm, aux_norm.max(), dtype=np.float32)

            elif normtype == 'zscore':
                # Normalizar datos utilizando Z-Score:
                image_data_norm = np.divide((image_data - np.mean(image_data)), np.std(image_data), dtype=np.float32)

            elif normtype == 'min-max':
                # Normalizar datos utilizando Norm min-max (0-1):
                aux_norm_1 = image_data - image_data.min()
                aux_norm_2 = image_data.max() - image_data.min()
                image_data_norm = np.divide(aux_norm_1, aux_norm_2, dtype=np.float32)
                
            elif normtype == 'menos1_1':
                # Normalizar datos utilizando Norm min-max (-1 , 1):
                aux_norm_1 = image_data - image_data.min()
                aux_norm_2 = image_data.max() - image_data.min()
                image_data_norm = 2 * np.divide(aux_norm_1, aux_norm_2, dtype=np.float32) - 1
                
            else:
                print("Nortype error - exit")
                return
                
            images_data_norm_by_sim.append(image_data_norm)

            if(t == (time_step-1) and pnt): 
                print(f'IMAGEN ORIGINAL')
                plotImage(image_data)
                print(f'IMAGEN NORMALIZADA')
                print(image_data_norm.shape)
                plotImage(image_data_norm)
                print(f'std: {np.std(image_data)} and mean: {np.mean(image_data)} of data image')
                print(f'std: {np.std(image_data_norm)} and mean: {np.mean(image_data_norm)} of normalizate data image')
                print("\n\n")
    
        images_data_norm_tot.append(images_data_norm_by_sim)
        
        ###VELOCITY###
        # Ruta donde estan guardadas las velocidades
        file_path_field = f'{base_path_field}/sim_{(sim+offset):01d}/'

        # Obtener velocidades del gas (gas1) del ultimo paso de tiempo (cambiar función a gusto)
        vfield_data_x, vfield_data_y, coordinates_x, coordinates_y = proyectar_velocidades_fargo(file_path_field, v_time, npix, incl, rot)
    
        vfield_data_coordinates_par = []
    
        #--------------------------vx-vy-------------------------------------
        if normtype == 'percentil':
            # Normalizar datos utilizando percentiles:
            p2_vx, p98_vx = np.percentile(vfield_data_x, (2, 98))
            vfield_data_norm_x = np.clip((vfield_data_x - p2_vx) / (p98 - p2_vx), 0, 1)

            p2_vy, p98_vy = np.percentile(vfield_data_y, (2, 98))
            vfield_data_norm_y = np.clip((vfield_data_y - p2_vy) / (p98 - p2_vy), 0, 1)

        elif normtype == 'log':
            # Normalizar datos utilizando transformación logarítmica:
            aux_norm_vx = np.log1p(vfield_data_x - vfield_data_x.min())
            vfield_data_norm_x = np.divide(aux_norm_vx, aux_norm_vx.max(), dtype=np.float32)

            aux_norm_vy = np.log1p(vfield_data_y - vfield_data_y.min())
            vfield_data_norm_y = np.divide(aux_norm_vy, aux_norm_vy.max(), dtype=np.float32)

        elif normtype == 'zscore':
            # Normalizar datos utilizando Z-Score:
            vfield_data_norm_x = np.divide((vfield_data_x - np.mean(vfield_data_x)), np.std(vfield_data_x), dtype=np.float32)
            
            vfield_data_norm_y = np.divide((vfield_data_y - np.mean(vfield_data_y)), np.std(vfield_data_y), dtype=np.float32)

        elif normtype == 'min-max':
            # Normalizar datos utilizando Norm min-max (0-1)
            aux_norm_1 = vfield_data_x - vfield_data_x.min()
            aux_norm_2 = vfield_data_x.max() - vfield_data_x.min()
            vfield_data_norm_x = np.divide(aux_norm_1, aux_norm_2, dtype=np.float32)

            aux_norm_1 = vfield_data_y - vfield_data_y.min()
            aux_norm_2 = vfield_data_y.max() - vfield_data_y.min()
            vfield_data_norm_y = np.divide(aux_norm_1, aux_norm_2, dtype=np.float32)

        elif normtype == 'menos1_1':
            # Normalizar datos utilizando Norm min-max (-1 , 1)
            aux_norm_1 = vfield_data_x - vfield_data_x.min()
            aux_norm_2 = vfield_data_x.max() - vfield_data_x.min()
            vfield_data_norm_x = 2 * np.divide(aux_norm_1, aux_norm_2, dtype=np.float32) - 1

            aux_norm_1 = vfield_data_y - vfield_data_y.min()
            aux_norm_2 = vfield_data_y.max() - vfield_data_y.min()
            vfield_data_norm_y = 2 * np.divide(aux_norm_1, aux_norm_2, dtype=np.float32) - 1
        
        
        else:
            print("Nortype error - exit")
            return
        
        vfield_data_norm_x_tot.append(vfield_data_norm_x)
    
        vfield_data_norm_y_tot.append(vfield_data_norm_y)
    
        if pnt:
            plotVelocity(vfield_data_norm_x, vfield_data_norm_y, 'Componente de Velocidad Horizontal Normalizada', \
                     'Componente de Velocidad Vertical Normalizada')
            print(f'std: {np.std(vfield_data_x)} and mean: {np.mean(vfield_data_x)} of data vx')
            print(f'std: {np.std(vfield_data_y)} and mean: {np.mean(vfield_data_y)} of data vy')
            print(f'std: {np.std(vfield_data_norm_x)} and mean: {np.mean(vfield_data_norm_x)} of normalizate data vx')
            print(f'std: {np.std(vfield_data_norm_y)} and mean: {np.mean(vfield_data_norm_y)} of normalizate data vy')
    
        vfield_data_coordinates_par.append(coordinates_x)
        vfield_data_coordinates_par.append(coordinates_y)
        vfield_data_coordinates_tot.append(vfield_data_coordinates_par)
        
    images_data_norm_tot_np = np.asarray(images_data_norm_tot)
    vfield_data_norm_x_tot_np = np.asarray(vfield_data_norm_x_tot)
    vfield_data_norm_y_tot_np = np.asarray(vfield_data_norm_y_tot)
    vfield_data_coordinates_tot_np = np.asarray(vfield_data_coordinates_tot)
    tag_data_np = np.array(tag_data, dtype=str)

    return images_data_norm_tot_np, vfield_data_norm_x_tot_np, vfield_data_norm_y_tot_np, vfield_data_coordinates_tot_np, tag_data_np


In [ ]:
pnt = False

In [ ]:
#__________________________________GENERAL SETTING___________________________________

##IMAGES SETTING###
npix = 128 # Tamaño de pixel
zscore = 'zscore'
min_max = 'min-max'
norm_menos1_1 = 'menos1_1'

normtype = norm_menos1_1
fieldtype = 'cartesian'
num_data = 5
output_np_file_name_img = f'./numpyFile_sim_images_{normtype}_{fieldtype}_v{num_data}'
output_np_file_name_tag = f'./numpyFile_sim_tag_{normtype}_{fieldtype}_v{num_data}'

##VELOCITY SETTING###
time_step = 20 #20 # Numero de imagenes por disco simulado 
v_time = 19 #19 # Indice de la salida por simulacion de fargo para obtener el campo de velocidades (19 last) - Obtengo el campo del ultimo tiempo.
output_np_file_name_vel_x = f'./numpyFile_sim_velocity_x_{normtype}_{fieldtype}_v{num_data}'
output_np_file_name_vel_y = f'./numpyFile_sim_velocity_y_{normtype}_{fieldtype}_v{num_data}'
output_np_file_name_coordinates_vel = f'./numpyFile_sim_velocity_coordinates_{normtype}_{fieldtype}_v{num_data}'

In [ ]:
# Definir indice de archivos con errores por configuración
err_sim_l1 = [0, 1, 2, 3, 4, 5, 6, 7]
err_sim_l2 = [10, 11, 12, 13, 14, 15]
err_sim_l3 = [20, 21, 22, 23, 24, 25]
err_sim_l4 = [30, 31, 32, 33, 34, 35]
err_sim_l6 = [40, 41, 42, 43, 44, 45]

In [ ]:
soft_limit, hard_limit = resource.getrlimit(resource.RLIMIT_NOFILE)
print(f"Límite de archivos abiertos: {soft_limit}/{hard_limit}")
resource.setrlimit(resource.RLIMIT_NOFILE, (4096, hard_limit))
gc.collect()


In [ ]:
# Configuraciones de las diferentes inclinacion y rotaciones o carpetas
configuraciones = [
    {   #RT
        'path_time': '/srv/nas02/alma/Radmc_data/out_leia_v2',
        'file_time': 'All_sim_pix128',
        'sim': 2000,
        'offset': 0,
        'path_field': '/srv/nas02/alma/Fargo_data/outputs/out_leia_v2',
        'incl': 0,
        'rot': 0,
        'err': err_sim_l1,
        'origen': 'leia'
    },
    {   #RT_2
        'path_time': '/srv/nas02/alma/Radmc_data/out_leia_v2',
        'file_time': 'All_sim_pix128_incl30_rot45',
        'sim': 2000,
        'offset': 0,
        'path_field': '/srv/nas02/alma/Fargo_data/outputs/out_leia_v2',
        'incl': 30,
        'rot': 45,
        'err': err_sim_l2,
        'origen': 'leia'
    },
    {   #RT_3
        'path_time': '/srv/nas02/alma/Radmc_data/out_leia_v2',
        'file_time': 'All_sim_pix128_incl60_rot215',
        'sim': 2000,
        'offset': 0,
        'path_field': '/srv/nas02/alma/Fargo_data/outputs/out_leia_v2',
        'incl': 60,
        'rot': 215,
        'err': err_sim_l3,
        'origen': 'leia'
    },
    {   #RT_4
        'path_time': '/srv/nas02/alma/Radmc_data/out_leia_v2',
        'file_time': 'All_sim_pix128_incl75_rot135',
        'sim': 2000,
        'offset': 0,
        'path_field': '/srv/nas02/alma/Fargo_data/outputs/out_leia_v2',
        'incl': 75,
        'rot': 135,
        'err': err_sim_l4,
        'origen': 'leia'
    },
    {   #RT_6
        'path_time': '/srv/nas02/alma/Radmc_data/out_leia_v2',
        'file_time': 'All_sim_pix128_incl50_rot315',
        'sim': 2000,
        'offset': 0,
        'path_field': '/srv/nas02/alma/Fargo_data/outputs/out_leia_v2',
        'incl': 50,
        'rot': 315,
        'err': err_sim_l6,
        'origen': 'leia'
    },
]

all_images = []
all_vfield_x = []
all_vfield_y = []
all_vfield_coords = []
all_tags = []

for i, config in enumerate(configuraciones):

    print(f"\n==== Procesando configuración l{i+1}: incl={config['incl']}, rot={config['rot']} ====")

    images_np, vfield_x_np, vfield_y_np, coords_np, tags_np = get_numpy_files_norm(
        config['sim'], config['offset'], time_step,
        npix, config['err'], v_time,
        config['path_time'], config['file_time'],
        config['path_field'], normtype,
        config['incl'], config['rot'], config['origen']
    )

    print(f"Imágenes shape: {images_np.shape}")
    print(f"Velocidad X shape: {vfield_x_np.shape}")
    print(f"Velocidad Y shape: {vfield_y_np.shape}")
    print(f"Coordenadas shape: {coords_np.shape}")
    print(f"Tags shape: {tags_np.shape}")

    all_images.append(images_np)
    all_vfield_x.append(vfield_x_np)
    all_vfield_y.append(vfield_y_np)
    all_vfield_coords.append(coords_np)
    all_tags.append(tags_np)

combined_images = np.concatenate(all_images, axis=0)
combined_vfield_x = np.concatenate(all_vfield_x, axis=0)
combined_vfield_y = np.concatenate(all_vfield_y, axis=0)
combined_vfield_coordinates = np.concatenate(all_vfield_coords, axis=0)
combined_tags = np.concatenate(all_tags, axis=0)

In [ ]:
print("\n===== SAVING numpy files All combinados =====")
print(combined_images.shape)
print(combined_vfield_x.shape)
print(combined_vfield_y.shape)
print(combined_vfield_coordinates.shape)
print(combined_tags.shape)

np.save(output_np_file_name_img, combined_images)
np.save(output_np_file_name_vel_x, combined_vfield_x)
np.save(output_np_file_name_vel_y, combined_vfield_y)
np.save(output_np_file_name_coordinates_vel, combined_vfield_coordinates)
np.save(output_np_file_name_tag, combined_tags)

In [ ]:
# Agregar nuevos archivos a los existentes (hacer más rápida la lectura)
base_file = 3
new_file = 4

# Cargar los archivos base
images = np.load(f'numpyFile_sim_images_menos1_1_cartesian_{base_file}.npy')
vel_x = np.load(f'numpyFile_sim_velocity_x_menos1_1_cartesian_{base_file}.npy')
vel_y = np.load(f'numpyFile_sim_velocity_y_menos1_1_cartesian_{base_file}.npy')
coords = np.load(f'numpyFile_sim_velocity_coordinates_menos1_1_cartesian_{base_file}.npy')
tags = np.load(f'numpyFile_sim_tag_menos1_1_cartesian_{base_file}.npy')

print("Original shapes:")
print("images.shape:", images.shape)
print("vel_x.shape:", vel_x.shape)
print("vel_y.shape:", vel_y.shape)
print("coords.shape:", coords.shape)
print("tags.shape:", tags.shape)

# Cargar los nuevos archivos
new_images = np.load(f'numpyFile_sim_images_menos1_1_cartesian_{new_file}.npy')
new_vel_x = np.load(f'numpyFile_sim_velocity_x_menos1_1_cartesian_{new_file}.npy')
new_vel_y = np.load(f'numpyFile_sim_velocity_y_menos1_1_cartesian_{new_file}.npy')
new_coords = np.load(f'numpyFile_sim_velocity_coordinates_menos1_1_cartesian_{new_file}.npy')
new_tags = np.load(f'numpyFile_sim_tag_menos1_1_cartesian_{new_file}.npy')

print("\nNew data shapes:")
print("new_images.shape:", new_images.shape)
print("new_vel_x.shape:", new_vel_x.shape)
print("new_vel_y.shape:", new_vel_y.shape)
print("new_coords.shape:", new_coords.shape)
print("new_tags.shape:", new_tags.shape)

In [ ]:
# Concatenar los datos
images = np.concatenate((images, new_images), axis=0)
vel_x = np.concatenate((vel_x, new_vel_x), axis=0)
vel_y = np.concatenate((vel_y, new_vel_y), axis=0)
coords = np.concatenate((coords, new_coords), axis=0)
tags = np.concatenate((tags, new_tags), axis=0)

print("\nConcatenated shapes:")
print("images.shape:", images.shape)
print("vel_x.shape:", vel_x.shape)
print("vel_y.shape:", vel_y.shape)
print("coords.shape:", coords.shape)
print("tags.shape:", tags.shape)

# Guardar los archivos combinados sobrescribiendo el archivo base
np.save(f'numpyFile_sim_images_menos1_1_cartesian_{base_file}.npy', images)
np.save(f'numpyFile_sim_velocity_x_menos1_1_cartesian_{base_file}.npy', vel_x)
np.save(f'numpyFile_sim_velocity_y_menos1_1_cartesian_{base_file}.npy', vel_y)
np.save(f'numpyFile_sim_velocity_coordinates_menos1_1_cartesian_{base_file}.npy', coords)
np.save(f'numpyFile_sim_tag_menos1_1_cartesian_{base_file}.npy', tags)

print("\nArchivos combinados y guardados exitosamente.")

In [ ]:
base_file = 3

# Cargar los archivos base
images = np.load(f'numpyFile_sim_images_menos1_1_cartesian_{base_file}.npy')
vel_x = np.load(f'numpyFile_sim_velocity_x_menos1_1_cartesian_{base_file}.npy')
vel_y = np.load(f'numpyFile_sim_velocity_y_menos1_1_cartesian_{base_file}.npy')
coords = np.load(f'numpyFile_sim_velocity_coordinates_menos1_1_cartesian_{base_file}.npy')
tags = np.load(f'numpyFile_sim_tag_menos1_1_cartesian_{base_file}.npy')

print("Original shapes:")
print("images.shape:", images.shape)
print("vel_x.shape:", vel_x.shape)
print("vel_y.shape:", vel_y.shape)
print("coords.shape:", coords.shape)
print("tags.shape:", tags.shape)

## Normalización completa

In [ ]:
# Normalizacion completa, considerando todos los datos (Se probo pero se descarto) 
combined_images = images
combined_vfield_x = vel_x
combined_vfield_y = vel_y

if normtype == 'min-max':
    print("-------NORMALIZANDO MIN_MAX (0-1)-------")
    # Normalización usando los valores min-max globales (0-1):
    # Images:
    aux_norm_1 = combined_images - combined_images.min()
    aux_norm_2 = combined_images.max() - combined_images.min()
    combined_images_norm = np.divide(aux_norm_1, aux_norm_2, dtype=np.float32)
    # Vx:
    aux_norm_x_1 = combined_vfield_x - combined_vfield_x.min()
    aux_norm_x_2 = combined_vfield_x.max() - combined_vfield_x.min()
    combined_vfield_x_norm = np.divide(aux_norm_x_1, aux_norm_x_2, dtype=np.float32)
    # Vy:
    aux_norm_y_1 = combined_vfield_y - combined_vfield_y.min()
    aux_norm_y_2 = combined_vfield_y.max() - combined_vfield_y.min()
    combined_vfield_y_norm = np.divide(aux_norm_y_1, aux_norm_y_2, dtype=np.float32)

elif normtype == 'percentil':
    print("-------NORMALIZANDO PERCENTIL-------")
    # Normalizar datos utilizando percentiles:
    # Images:
    p2, p98 = np.percentile(combined_images, (2, 98))
    combined_images_norm = np.clip((combined_images - p2) / (p98 - p2), 0, 1)
    # Vx:
    p2_vx, p98_vx = np.percentile(combined_vfield_x, (2, 98))
    combined_vfield_x_norm = np.clip((combined_vfield_x - p2_vx) / (p98 - p2_vx), 0, 1)
    # Vy:
    p2_vy, p98_vy = np.percentile(combined_vfield_y, (2, 98))
    combined_vfield_y_norm = np.clip((combined_vfield_y - p2_vy) / (p98 - p2_vy), 0, 1)

elif normtype == 'log':
    print("-------NORMALIZANDO LOG-------")
    # Normalizar datos utilizando transformación logarítmica:
    # Images:
    aux_norm = np.log1p(combined_images - combined_images.min())
    combined_images_norm = np.divide(aux_norm, aux_norm.max(), dtype=np.float32)
    # Vx:
    aux_norm_vx = np.log1p(combined_vfield_x - combined_vfield_x.min())
    combined_vfield_x_norm = np.divide(aux_norm_vx, aux_norm_vx.max(), dtype=np.float32)
    # Vy:
    aux_norm_vy = np.log1p(combined_vfield_y - combined_vfield_y.min())
    combined_vfield_y_norm = np.divide(aux_norm_vy, aux_norm_vy.max(), dtype=np.float32)

elif normtype == 'zscore':
    print("-------NORMALIZANDO ZSCORE-------")
    # Normalizar datos utilizando Z-Score:
    # Images:
    combined_images_norm = np.divide((combined_images - np.mean(combined_images)), np.std(combined_images), dtype=np.float32)
    # Vx:
    combined_vfield_x_norm = np.divide((combined_vfield_x - np.mean(combined_vfield_x)), np.std(combined_vfield_x), dtype=np.float32)
    # Vy:
    combined_vfield_y_norm = np.divide((combined_vfield_y - np.mean(combined_vfield_y)), np.std(combined_vfield_y), dtype=np.float32)

elif normtype == 'menos1_1':
    print("-------NORMALIZANDO -1 a 1-------")
    # Normalización usando los valores min-max globales (0-1):
    # Images:
    aux_norm_1 = combined_images - combined_images.min()
    aux_norm_2 = combined_images.max() - combined_images.min()
    combined_images_norm = 2 * np.divide(aux_norm_1, aux_norm_2, dtype=np.float32) - 1
    # Vx:
    aux_norm_x_1 = combined_vfield_x - combined_vfield_x.min()
    aux_norm_x_2 = combined_vfield_x.max() - combined_vfield_x.min()
    combined_vfield_x_norm = 2 * np.divide(aux_norm_x_1, aux_norm_x_2, dtype=np.float32) - 1
    # Vy:
    aux_norm_y_1 = combined_vfield_y - combined_vfield_y.min()
    aux_norm_y_2 = combined_vfield_y.max() - combined_vfield_y.min()
    combined_vfield_y_norm = 2 * np.divide(aux_norm_y_1, aux_norm_y_2, dtype=np.float32) - 1

elif normtype == 'transformer_torch':
    print("-------NORMALIZANDO torch transfor-------")
    
    
else: 
    print("Error normtype - data sin normalizar")
    combined_images_norm = combined_images
    combined_vfield_x_norm = combined_vfield_x
    combined_vfield_y_norm = combined_vfield_y


#Probar normalizacion:
id = 0
print(f'IMAGEN ORIGINAL')
plotImage(combined_images_norm[id, 18])
print(f'VELOCIDADES NORMALIZADAS')
plotVelocity(combined_vfield_x_norm[id], combined_vfield_y_norm[id], 'Componente de Velocidad Horizontal Cartesiana Normalizada', \
                     'Componente de Velocidad Vertical Cartesiana Normalizada')

In [ ]:
print("\n ===== SAVING numpy files All")
print(combined_images_norm.shape)
print(combined_vfield_x_norm.shape)
print(combined_vfield_y_norm.shape)

np.save(output_np_file_name_img, combined_images_norm)
np.save(output_np_file_name_vel_x, combined_vfield_x_norm)
np.save(output_np_file_name_vel_y, combined_vfield_y_norm)